In [1]:
# Pradėta 2024-11-21_ObjectProperties.ipynb
# Tęsinys. Čia 2024-11-28_varle_lt.ipynb

item_type = 'mobilieji-telefonai'
# 'dronai', 'televizoriai', 'saldytuvai', 'televizoriai'

import pandas as pd
import time
import sqlite3

from bs4 import BeautifulSoup

# Selenium
import selenium
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium import webdriver

from my_lib.properties_of import properties_of
from my_lib.my_db_functions import save_df_to_sqlite

opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
driver = webdriver.Chrome(options=opcijos)
print('WebDriver started.',end=' ')

url = f'https://www.varle.lt/{item_type}/'
driver.get(url)
time.sleep(3)
source = driver.page_source
print(f'Prskaityta {url}.')

bs = BeautifulSoup(source,'html.parser')
pages_numbers = bs.find('div',{'class':'PAGINATION'}).find_all('li',{'class':'number'})
total_pages = int(pages_numbers[-1].text.strip())
print(f'Total pages: {total_pages}')

items_df = pd.DataFrame(columns=('id','price','url','price_in_cents')) # šios programos rezultatas

WebDriver started. Prskaityta https://www.varle.lt/mobilieji-telefonai/.
Total pages: 59


In [2]:
# Parsinamas pirmasis prekių sąrašo puslapis.

items_blocks = bs.find_all('div',{'class':'GRID_ITEM'})
for item_block in items_blocks:

    item_href = item_block.find('div',{'class':'product-title'}).find('a')['href']
    item_url = 'https://www.varle.lt'+ item_href
    item_id = item_href.split('-')[-1].split('.')[0]

    price_container = item_block.find('div',{'class':'price-container'})
    price_value = price_container.find('span',{'class':'price-value'}).getText()
    item_price = ''.join(price_value.split())

    items_df.loc[len(items_df)] = {'id':int(item_id),'price':item_price,'url':item_url}

print(len(items_df))

38


In [3]:
# Perskaitomi kiti prekių sąrašo puslapiai,
# po to įrašoma į db

for page_number in range(2,total_pages+1):

    url = f'https://www.varle.lt/{item_type}/?p={page_number}'
    driver.get(url)
    time.sleep(60)
    source = driver.page_source
    print(f'Prskaityta {url}')

    bs = BeautifulSoup(source,'html.parser')

    items_blocks = bs.find_all('div',{'class':'GRID_ITEM'})
    for item_block in items_blocks:
        item_href = item_block.find('div',{'class':'product-title'}).find('a')['href']
        item_url = 'https://www.varle.lt'+item_href
        item_id = item_href.split('-')[-1].split('.')[0]

        price_container = item_block.find('div',{'class':'price-container'})
        price_value = price_container.find('span',{'class':'price-value'}).getText()
        item_price = ''.join(price_value.split())

        items_df.loc[len(items_df)] = {'id':int(item_id),'price':item_price,'url':item_url}

    items_total = len(items_df)
    items_df.drop_duplicates(subset='id',keep='first',inplace=True)
    print(f'Get {items_total} items. Unique {len(items_df)} items.')

driver.close()


# Pridedamas kainos stulpelis centais (int)
import re

def price_in_cents(price:str) -> int:
    # Pašalina visus simbolius, išskyrus skaičius, kablelį ir tašką:
    cleaned = re.sub(r'[^\d.,]', '', price).replace(',', '.').replace(' ', '')
    return int(float(cleaned)*100)

items_df['price_in_cents'] = items_df['price'].apply(price_in_cents)


# Save to SQLite3
save_df_to_sqlite(items_df,"./my_scrap_working_data.db",table_name=item_type)


items_df

KeyboardInterrupt: 